In [5]:
import glob
import os
import pandas as pd
import geopy.distance
pd.set_option('display.max_columns', None)  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def reduce_memory(df):
    for c in df.columns:
        if df[c].dtype=='int':
            if df[c].min()<0:
                if df[c].abs().max()<2**7:
                    df[c] = df[c].astype('int8')
                elif df[c].abs().max()<2**15:
                    df[c] = df[c].astype('int16')
                elif df[c].abs().max()<2**31:
                    df[c] = df[c].astype('int32')
                else:
                    continue
            else:
                if df[c].max()<2**8:
                    df[c] = df[c].astype('uint8')
                elif df[c].max()<2**16:
                    df[c] = df[c].astype('uint16')
                elif df[c].max()<2**32:
                    df[c] = df[c].astype('uint32')
                else:
                    continue
    return df

In [7]:
#check the data structure of each file
for files in glob.glob("C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata/*.csv"):
    df = pd.read_csv(files)
    print (files, 'has ', len(df.columns), 'columns\n')
    print (df.columns, '\n')
    print (df.dtypes, '\n')

C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-1990_to_1999.csv has  10 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price             int64
dtype: object 

C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-2000_to_feb2012.csv has  10 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object

Jan 2015 onwards dataset has 11 columns. Extra column called remaininglease

In [154]:
df = pd.DataFrame()
for files in glob.glob("C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata/*.csv"):
    print ('Concatenating now...', files)
    df = pd.concat([df,pd.read_csv(files).iloc[:,:]],axis=0)
print ('Datasets are concatenated into one file.')

df.to_csv("C:/Users/weiji/OneDrive/Desktop/hdbdata/resale-flat-merged.csv", encoding='utf-8')

Concatenating now... C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-1990_to_1999.csv
Concatenating now... C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-2000_to_feb2012.csv
Concatenating now... C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-jan2015_onwards.csv


C:\Users\weiji\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


Concatenating now... C:/Users/weiji/OneDrive/Desktop/hdbdata/rawdata\resale-flat-mar2012_to_dec-2014.csv
Datasets are concatenated into one file.


# Display Dataframe

In [11]:
df.shape

(789098, 11)

In [13]:
df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO
1,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO
2,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO
3,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO
4,216,NEW GENERATION,3 ROOM,73.0,1976,1990-01,NaN,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO


In [14]:
df.dtypes

block                   object
flat_model              object
flat_type               object
floor_area_sqm         float64
lease_commence_date      int64
month                   object
remaining_lease        float64
resale_price           float64
storey_range            object
street_name             object
town                    object
dtype: object

As One MAP's API requires the address of the 'searchval' to be in order of 'block' + 'street_name', will have to combine these 2 variables

In [15]:
#combine the block and street name to get full address of flats
df["full_address"] = df["block"].map(str) + ' ' + df["street_name"].map(str)


In [16]:
#check for first 10 rows
df.head(10)

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town,full_address
0,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,9000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
1,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
2,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,8000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
3,309,IMPROVED,1 ROOM,31.0,1977,1990-01,NaN,6000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO,309 ANG MO KIO AVE 1
4,216,NEW GENERATION,3 ROOM,73.0,1976,1990-01,NaN,47200.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO,216 ANG MO KIO AVE 1
5,211,NEW GENERATION,3 ROOM,67.0,1977,1990-01,NaN,46000.0,01 TO 03,ANG MO KIO AVE 3,ANG MO KIO,211 ANG MO KIO AVE 3
6,202,NEW GENERATION,3 ROOM,67.0,1977,1990-01,NaN,42000.0,07 TO 09,ANG MO KIO AVE 3,ANG MO KIO,202 ANG MO KIO AVE 3
7,235,NEW GENERATION,3 ROOM,67.0,1977,1990-01,NaN,38000.0,10 TO 12,ANG MO KIO AVE 3,ANG MO KIO,235 ANG MO KIO AVE 3
8,235,NEW GENERATION,3 ROOM,67.0,1977,1990-01,NaN,40000.0,04 TO 06,ANG MO KIO AVE 3,ANG MO KIO,235 ANG MO KIO AVE 3
9,232,NEW GENERATION,3 ROOM,67.0,1977,1990-01,NaN,47000.0,01 TO 03,ANG MO KIO AVE 3,ANG MO KIO,232 ANG MO KIO AVE 3


looking through the data, there are duplicate addresses. To reduce the number of API calls, could use a list of unique addresses

In [20]:
# Obtain the number of unique full addresses
full_address_list = df["full_address"].unique().tolist()
len(full_address_list)


8904

# Batch Geocoding to get Lat and Long

In order to obtain the lattitude and longitude, need to query ONE MAP API. 
One MAP's search function returns search results with both latitude, longitude and x, y coordinates of the searched. 
One MAP client SDK has some available python code for use. However, there were problems with installing the swagger_client package. As such, will attempt to use the requests package as it should be able to handle rest APIs.

to access the One Map API, need to have an postman token (i.e., access token). will have to create one map developer account and install the postman app.

In [22]:
%%time
import requests

query_results = []
querystring = {}

for x in full_address_list:
    url = "https://developers.onemap.sg/commonapi/search"
    

    querystring.update({"searchVal":x,"returnGeom":"Y","getAddrDetails":"Y","pageNum":"1"})
    #print (querystring['searchVal'])
    
    payload = "------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"searchVal\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"returnGeom {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"getAddrDetails {Y/N}\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW\r\nContent-Disposition: form-data; name=\"pageNum\"\r\n\r\n\r\n------WebKitFormBoundary7MA4YWxkTrZu0gW--"
    headers = {
    'content-type': "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    'Cache-Control': "no-cache",
    'Postman-Token': "2c00c356-6651-356f-037c-81f92877866c"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    

    query_results.append(response.json())

print('Query Complete.')

Query Complete.
Wall time: 14min 30s


In [24]:
df1 = pd.DataFrame.from_dict(query_results[0]['results'])


In [25]:
#save results into dataframe

latnlong_df = pd.DataFrame()
for entry in range(len(query_results)):
    temp_df = pd.DataFrame.from_dict(query_results[entry]['results'])
    latnlong_df = latnlong_df.append(temp_df)

In [26]:
latnlong_df.head()

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y
0,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,216,ANG MO KIO 22,1.36619678831055,103.841505011903,103.841505011903,560216,ANG MO KIO AVENUE 1,ANG MO KIO 22,28911.0522399394,38692.6167905741
0,211 ANG MO KIO AVENUE 3 HDB-ANG MO KIO SINGAPO...,211,HDB-ANG MO KIO,1.36919696561704,103.841666636086,103.841666636086,560211,ANG MO KIO AVENUE 3,HDB-ANG MO KIO,28929.0379298831,39024.3611867
0,202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAP...,202,ANG MO KIO VIEW,1.3684464401094,103.844516260527,103.844516260527,560202,ANG MO KIO AVENUE 3,ANG MO KIO VIEW,29246.1670971225,38941.3731856439
1,202 ANG MO KIO AVENUE 3 OCBC Ang Mo Kio Ave 3 ...,202,OCBC Ang Mo Kio Ave 3,1.36845752328242,103.844515336938,103.844515336938,560202,ANG MO KIO AVENUE 3,OCBC Ang Mo Kio Ave 3,29246.0643070503,38942.5987060368
0,235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...,235,KEBUN BARU PALM VIEW,1.36682360872345,103.83649123351,103.83649123351,560235,ANG MO KIO AVENUE 3,KEBUN BARU PALM VIEW,28353.0784864312,38761.926084429


In [27]:
latnlong_df["full_address"] = latnlong_df["BLK_NO"].map(str) + ' ' + latnlong_df["ROAD_NAME"].map(str)


In [29]:
latnlong_df["lat_long"] = latnlong_df["LATITUDE"] +' '+ latnlong_df["LONGITUDE"]
# convert lat_long column to tuples so that we can use it for calculation of distance later
latnlong_df["lat_long"] = latnlong_df["lat_long"].apply(lambda x: tuple(x.split(' ')))

In [31]:

# Save/load latnlong_df
latnlong_df.to_pickle('C:/Users/weiji/OneDrive/Desktop/hdbdata/latnlong_df.pkl')    
#latnlong_df = pd.read_pickle('data/latnlong_df.pkl')

# Obtain Distance of MRT station from address


In [72]:
# MRT station data from data.gov.sg is in SHP format
# Use shapefile from pyshp package to read the data into pandas; 
# see https://www.adamerispaha.com/2017/01/24/reading-shapefiles-into-pandas-dataframes/

def read_shapefile(shp_path):

    import shapefile

    myshp = open(shp_path)
    #read file, parse out the records and shapes
    sf = shapefile.Reader(shp_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    
    #for each item in records, it is showing a list of records, need to change it to a list of list
    records = [list(item) for item in records]
    df = pd.DataFrame(columns=fields, data=records)

    return df

In [71]:
#read the datafile
mrt_latlong_df = read_shapefile("C:/Users/weiji/OneDrive/Desktop/sdcp-mrt-station-point-shp/MRT_PT.shp")
mrt_latlong_df.head()

,OBJECTID,UNIQUE_ID,REMARKS,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR
0,1,21,,52AA2BBA,2011-03-29,30170.9683,29101.6566
1,2,2,,13C79F9F4B8E6DDF,2011-03-29,28643.7989,31399.5329
2,3,3,,E7FAF621B9D811A6,2011-03-29,27879.1516,31804.3462
3,4,4,,4EA0DD3BD9F8DEBC,2011-03-29,28516.7988,32725.0980
4,5,5,,7B9BB370D55B1AE4,2011-03-29,29842.3639,32177.4094


In [74]:
station_address_list = mrt_latlong_df[""].unique().tolist()
len(station_address_list)

KeyError: ''

In [147]:
latnlong_df

,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y,full_address,lat_long
0,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,216,ANG MO KIO 22,1.36619678831055,103.841505011903,103.841505011903,560216,ANG MO KIO AVENUE 1,ANG MO KIO 22,28911.0522399394,38692.6167905741,216 ANG MO KIO AVENUE 1,"(1.36619678831055, 103.841505011903)"
0,211 ANG MO KIO AVENUE 3 HDB-ANG MO KIO SINGAPO...,211,HDB-ANG MO KIO,1.36919696561704,103.841666636086,103.841666636086,560211,ANG MO KIO AVENUE 3,HDB-ANG MO KIO,28929.0379298831,39024.3611867,211 ANG MO KIO AVENUE 3,"(1.36919696561704, 103.841666636086)"
0,202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAP...,202,ANG MO KIO VIEW,1.3684464401094,103.844516260527,103.844516260527,560202,ANG MO KIO AVENUE 3,ANG MO KIO VIEW,29246.1670971225,38941.3731856439,202 ANG MO KIO AVENUE 3,"(1.3684464401094, 103.844516260527)"
1,202 ANG MO KIO AVENUE 3 OCBC Ang Mo Kio Ave 3 ...,202,OCBC Ang Mo Kio Ave 3,1.36845752328242,103.844515336938,103.844515336938,560202,ANG MO KIO AVENUE 3,OCBC Ang Mo Kio Ave 3,29246.0643070503,38942.5987060368,202 ANG MO KIO AVENUE 3,"(1.36845752328242, 103.844515336938)"
0,235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...,235,KEBUN BARU PALM VIEW,1.36682360872345,103.83649123351,103.83649123351,560235,ANG MO KIO AVENUE 3,KEBUN BARU PALM VIEW,28353.0784864312,38761.926084429,235 ANG MO KIO AVENUE 3,"(1.36682360872345, 103.83649123351)"
1,235 ANG MO KIO AVENUE 3 PCF SPARKLETOTS PRESCH...,235,PCF SPARKLETOTS PRESCHOOL @ YIO CHU KANG Blk 2...,1.36682360708104,103.83649123895,103.83649123895,560235,ANG MO KIO AVENUE 3,PCF SPARKLETOTS PRESCHOOL @ YIO CHU KANG Blk 2...,28353.0790917896,38761.9259028211,235 ANG MO KIO AVENUE 3,"(1.36682360708104, 103.83649123895)"
0,232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...,232,KEBUN BARU PALM VIEW,1.36834606813392,103.837196046409,103.837196046409,560232,ANG MO KIO AVENUE 3,KEBUN BARU PALM VIEW,28431.5154539891,38930.2720091306,232 ANG MO KIO AVENUE 3,"(1.36834606813392, 103.837196046409)"
0,308C ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGA...,308C,TECK GHEE VISTA,1.36527118786113,103.844818746979,103.844818746979,563308,ANG MO KIO AVENUE 1,TECK GHEE VISTA,29279.8318750454,38590.2701135632,308C ANG MO KIO AVENUE 1,"(1.36527118786113, 103.844818746979)"
1,308B ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGA...,308B,TECK GHEE VISTA,1.3652254590823,103.844427064658,103.844427064658,562308,ANG MO KIO AVENUE 1,TECK GHEE VISTA,29236.2423160665,38585.2134534009,308B ANG MO KIO AVENUE 1,"(1.3652254590823, 103.844427064658)"
2,308A ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGA...,308A,TECK GHEE VISTA,1.36547951592016,103.844086654914,103.844086654914,561308,ANG MO KIO AVENUE 1,TECK GHEE VISTA,29198.3586352326,38613.3055928405,308A ANG MO KIO AVENUE 1,"(1.36547951592016, 103.844086654914)"


In [151]:
# Save final data in csv file
df5 = latnlong_df
df5 = latnlong_df.to_csv("C:/Users/weiji/OneDrive/Desktop/hdbdata/latnlong_df.csv")